In [1]:
# forming the initial vocabulary on the training set of 600 samples
# Training set: 
# Test set: 
# Not using stemming currently, need to modify later

from collections import defaultdict
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn import linear_model
from sets import Set

comments = []
usernames = []
stop_words = set(stopwords.words('english'))
should_stem = 0
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

def stem(w):
    if(should_stem == 0):
        return w
    else:
        return stemmer.stem(w)
    
#creating a small test file for convenience
def read(f):
    for l in open(f):
        yield eval(l)
        
x = open("opt_dataset/opt.json")
i = 0
training = open("opt_dataset/small_opt.json",'w')

for l in x:
    if (i>=0) and (i<3000):
        training.write(l)
        if(i==0):
            print l
    i += 1
training.close()

for i in read('opt_dataset/small_opt.json'):
    comments.append(i['comment'])
    usernames.append(i['userName'])

        
# extracing all the known labels
labels = []
for l in open('opt_dataset/labels_1-700.txt'):
    if(eval(l) == 1):
        labels.append(1)
    else:
        labels.append(0)

{'userName': 'Roxanne Liu', 'recievedDate': '11/08/2015', 'docID': 'ICEB-2015-0002-12058', 'comment': 'It is very enssential for America to keep the international students. They are epquiped with skills and knowledge. The development of America relies on talents. Competition should not be the reason to ask them to leave.', 'postedDate': '11/11/2015'}



In [8]:
# Multinomial naive bayes!
def fit_params(num_examples, data_labels):
    train_no = num_examples
    print "entered fit_aprams"
    # extracting the vocab
    vocab = Set()
    for i in range(train_no):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        for w in r.split():
            if(w not in stop_words):
                w = stem(w)
                vocab.add(w)
                
    pos_wordCount_mult = defaultdict(int)
    neg_wordCount_mult = defaultdict(int)
    
    for w in vocab:
        pos_wordCount_mult[w] = 0
        neg_wordCount_mult[w] = 0
    numberpos_mult = 0.0
    numberneg_mult = 0.0
    posexamples = 0.0
    negexamples = 0.0
    # calculating the probabilites
    for i in range(train_no):
        if(data_labels[i] == 1):
            posexamples += 1
        else:
            negexamples += 1
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for w in bag:
            w = stem(w)
            if(w in vocab):
                if(data_labels[i] == 1):
                    pos_wordCount_mult[w] += 1
                    numberpos_mult += 1
                else:
                    neg_wordCount_mult[w] += 1
                    numberneg_mult += 1
                    
    posprob = posexamples/(posexamples + negexamples)
    negprob = negexamples/(posexamples + negexamples)
    
    pos_wordprob_mult = defaultdict(float)
    neg_wordprob_mult = defaultdict(float)

    for w in vocab:
        pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
        neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)
    
    return vocab, pos_wordprob_mult,neg_wordprob_mult,posprob, negprob

import math
def predpos_mult(words,vocab,prob, frac):
    sum = 0.0
    for w in words:
        w = stem(w)
        if(w in vocab):
            sum += math.log(prob[w])
    return (math.log(frac) + sum)
            
def predneg_mult(words, vocab, prob, frac):
    sum = 0.0
    for w in words:
        w = stem(w)
        if(w in vocab):
            sum += math.log(prob[w])
    return (math.log(frac) + sum)

def train_NB(num_examples,data_labels):
    vocab, pp, nn, p, n = fit_params(num_examples,data_labels)
    predictions_mult = []
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words,vocab,pp,p)
        y = predneg_mult(words,vocab,nn,n)
        if(x > y):
            predictions_mult.append(1)
        else:
            predictions_mult.append(0)
    return predictions_mult


In [28]:
# functions for accuracy prediction...
def print_accuracies(predictions, no_train,verbatim):
    train_accuracy = 0.0
    test_accuracy = 0.0
    rate = 0.0
    neg_examples = 0.0
    for i in range(700):
        if(i>=no_train and labels[i] == 0):
            neg_examples += 1
        if(predictions[i] == labels[i]):
            if(labels[i] == 0 and i>=no_train):
                rate += 1
            if i<no_train:
                train_accuracy += 1
            else:
                test_accuracy += 1
                
        elif(i>=no_train  and i<700 and verbatim == 1):
            r = ''.join([c for c in comments[i].lower() if not c in punctuation])
            words = r.split()
            x = predpos_mult(words) # Likelihood of positive
            y = predneg_mult(words) # Likelihood of negative
            print comments[i], "True label:",labels[i],"Predicted label:",predictions[i],"positive:",x,"negative:",y
    print "\n","Test accuracy:",test_accuracy/(700-no_train),"Train accuracy:", train_accuracy/no_train, "frac negatives identified:",(rate + 0.0001)/(neg_examples+0.0001)

In [34]:
predictions = train_NB(500,labels)
print_accuracies(predictions,500,0)

entered fit_aprams

Test accuracy: 0.88 Train accuracy: 0.992 frac negatives identified: 0.351353104451


In [35]:
# Semi supervised learning, Some kind of expectation maximization...
# Refer to https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Semi-supervised_parameter_estimation
# running the iterative process a fixed number of times.

num_iter = 0

while(num_iter < 6):
    predictions = train_NB(len(comments),predictions)
    print_accuracies(predictions,500,0)
    num_iter += 1
    print "iteration number:",num_iter

entered fit_aprams

Test accuracy: 0.91 Train accuracy: 0.976 frac negatives identified: 0.513514828338
iteration number: 1
entered fit_aprams

Test accuracy: 0.92 Train accuracy: 0.976 frac negatives identified: 0.567568736301
iteration number: 2
entered fit_aprams

Test accuracy: 0.92 Train accuracy: 0.978 frac negatives identified: 0.567568736301
iteration number: 3
entered fit_aprams

Test accuracy: 0.925 Train accuracy: 0.978 frac negatives identified: 0.594595690282
iteration number: 4
entered fit_aprams

Test accuracy: 0.925 Train accuracy: 0.978 frac negatives identified: 0.594595690282
iteration number: 5
entered fit_aprams

Test accuracy: 0.925 Train accuracy: 0.978 frac negatives identified: 0.594595690282
iteration number: 6
